In [7]:
#import needed libraries
import pandas as pd
import numpy as np

In [17]:
#makes sure that pandas shows all columns
pd.set_option('display.max_columns', None)

In [2]:
#download file into dataframe

filename = 'support2.csv'

df = pd.read_csv(filename)

In [3]:
df.shape

(9105, 47)

Dataset has 47 columns, 46 potential features

In [4]:
df.head()

,age,death,sex,hospdead,slos,d.time,dzgroup,dzclass,num.co,edu,...,crea,sod,ph,glucose,bun,urine,adlp,adls,sfdm2,adlsc
1,62.84998,0,male,0,5,2029,Lung Cancer,Cancer,0,11.0,...,1.199951,141.0,7.459961,NaN,NaN,NaN,7.0,7.0,NaN,7.0
2,60.33899,1,female,1,4,4,Cirrhosis,COPD/CHF/Cirrhosis,2,12.0,...,5.500000,132.0,7.250000,NaN,NaN,NaN,NaN,1.0,<2 mo. follow-up,1.0
3,52.74698,1,female,0,17,47,Cirrhosis,COPD/CHF/Cirrhosis,2,12.0,...,2.000000,134.0,7.459961,NaN,NaN,NaN,1.0,0.0,<2 mo. follow-up,0.0
4,42.38498,1,female,0,3,133,Lung Cancer,Cancer,2,11.0,...,0.799927,139.0,NaN,NaN,NaN,NaN,0.0,0.0,no(M2 and SIP pres),0.0
5,79.88495,0,female,0,16,2029,ARF/MOSF w/Sepsis,ARF/MOSF,1,NaN,...,0.799927,143.0,7.509766,NaN,NaN,NaN,NaN,2.0,no(M2 and SIP pres),2.0


In [21]:
df.describe(include = 'all')


,age,death,sex,hospdead,slos,d.time,dzgroup,dzclass,num.co,edu,income,scoma,charges,totcst,totmcst,avtisst,race,sps,aps,surv2m,surv6m,hday,diabetes,dementia,ca,prg2m,prg6m,dnr,dnrday,meanbp,wblc,hrt,resp,temp,pafi,alb,bili,crea,sod,ph,glucose,bun,urine,adlp,adls,sfdm2,adlsc
count,9105.000000,9105.000000,9105,9105.000000,9105.000000,9105.000000,9105,9105,9105.000000,7471.000000,6123,9104.000000,8.933000e+03,8217.000000,5630.000000,9023.000000,9063,9104.000000,9104.000000,9104.000000,9104.000000,9105.000000,9105.000000,9105.000000,9105,7456.000000,7472.000000,9075,9075.000000,9104.000000,8893.000000,9104.000000,9104.000000,9104.000000,6780.000000,5733.000000,6504.000000,9038.000000,9104.000000,6821.000000,4605.000000,4753.000000,4243.000000,3464.000000,6238.000000,7705,9105.000000
unique,NaN,NaN,2,NaN,NaN,NaN,8,4,NaN,NaN,4,NaN,NaN,NaN,NaN,NaN,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,NaN,NaN,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5,NaN
top,NaN,NaN,male,NaN,NaN,NaN,ARF/MOSF w/Sepsis,ARF/MOSF,NaN,NaN,under $11k,NaN,NaN,NaN,NaN,NaN,white,NaN,NaN,NaN,NaN,NaN,NaN,NaN,no,NaN,NaN,no dnr,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<2 mo. follow-up,NaN
freq,NaN,NaN,5125,NaN,NaN,NaN,3515,4227,NaN,NaN,2855,NaN,NaN,NaN,NaN,NaN,7191,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5995,NaN,NaN,5880,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3123,NaN
mean,62.650823,0.681054,NaN,0.259198,17.863042,478.449863,NaN,NaN,1.868644,11.747691,NaN,12.058546,5.999579e+04,30825.867768,28828.877838,22.610928,NaN,25.525872,37.597979,0.635870,0.520096,4.400000,0.195277,0.032510,NaN,0.618769,0.499679,NaN,14.582590,84.546408,12.347677,97.156711,23.330294,37.103341,239.529070,2.950520,2.554463,1.770961,137.568541,7.415364,159.873398,32.349463,2191.546047,1.157910,1.637384,NaN,1.888272
std,15.593710,0.466094,NaN,0.438219,22.006440,560.383272,NaN,NaN,1.344409,3.447743,NaN,24.636694,1.026488e+05,45780.820986,43604.261932,13.233248,NaN,9.899377,19.903852,0.248175,0.253343,9.126811,0.396436,0.177359,NaN,0.298367,0.304367,NaN,19.944216,27.687692,9.266329,31.559292,9.573801,1.251796,109.665593,0.875944,5.318448,1.686041,6.029326,0.080563,88.391541,26.792288,1455.245777,1.739672,2.231358,NaN,2.003763
min,18.041990,0.000000,NaN,0.000000,3.000000,3.000000,NaN,NaN,0.000000,0.000000,NaN,0.000000,1.169000e+03,0.000000,-102.719970,1.000000,NaN,0.199982,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,NaN,0.000000,0.000000,NaN,-88.000000,0.000000,0.000000,0.000000,0.000000,31.699220,12.000000,0.399963,0.099991,0.099991,110.000000,6.829102,0.000000,1.000000,0.000000,0.000000,0.000000,NaN,0.000000
25%,52.797000,0.000000,NaN,0.000000,6.000000,26.000000,NaN,NaN,1.000000,10.000000,NaN,0.000000,9.740000e+03,5929.566400,5177.404300,12.000000,NaN,19.000000,23.000000,0.507690,0.342957,1.000000,0.000000,0.000000,NaN,0.500000,0.200000,NaN,4.000000,63.000000,7.000000,72.000000,18.000000,36.195310,155.093750,2.399902,0.500000,0.899902,134.000000,7.379883,103.000000,14.000000,1165.500000,0.000000,0.000000,NaN,0.000000
50%,64.856990,1.000000,NaN,0.000000,11.000000,233.000000,NaN,NaN,2.000000,12.000000,NaN,0.000000,2.502400e+04,14452.734400,13223.500000,19.500000,NaN,23.898438,34.000000,0.715942,0.574951,1.000000,0.000000,0.000000,NaN,0.700000,0.500000,NaN,9.000000,77.000000,10.599609,100.000000,24.000000,36.695310,224.000000,2.899902,0.899902,1.199951,137.000000,7.419922,135.000000,23.000000,1968.000000,0.000000,1.000000,NaN,1.000000
75%,73.998960,1.000000,NaN,1.000000,20.000000,761.000000,NaN,NaN,3.000000,14.000000,NaN,9.000000,6.459800e+04,36087.937500,34223.601600,31.666656,NaN,30.199219,49.000000,0.825928,0.725952,3.000000,0.000000,0.000000,NaN,0.900000,0.750000,NaN,17.000000,107.000000,15.298828,120.000000,28.000000,38.195310,304.750000,3.599609,1.899902,1.899902,141.000000,7.469727,188.000000,42.000000,3000.000000,2.000000,3.000000,NaN,3.000000


In [23]:
summary = df.describe(include='all')

count_values = summary.loc['count']


"""columns where percent of missing values is more then 50%"""

#find a trashhold of 50%
half_rows = len(df)/2

columns_big_null_amount = count_values[count_values < half_rows ].index.tolist()
columns_big_null_amount


['urine', 'adlp']

Should check if columns 'urine' and 'adlp' have a big cirrelation to labels
'adlp' is Index of Activities of Daily Living (ADL) filled byt the patient, there also 'adlp' that doesn't have as much missing values, also means Index of Activities of Daily Living (ADL) but filled by surrogate (family member)